<pre>文本预处理

对于序列数据处理问题，我们在 8.1节中 评估了所需的统计工具和预测时面临的挑战。  
这样的数据存在许多种形式，文本是最常见例子之一。 例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。  
本节中，我们将解析文本的常见预处理步骤。 这些步骤通常包括：

    将文本作为字符串加载到内存中。

    将字符串拆分为词元（如单词和字符）。

    建立一个词表，将拆分的词元映射到数字索引。

    将文本转换为数字索引序列，方便模型操作。


读取数据集

首先，我们从H.G.Well的时光机器中加载文本。  
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀， 而现实中的文档集合可能会包含数十亿个单词。  
下面的函数将数据集读取到由多条文本行组成的列表中，其中每条文本行都是一个字符串。  
为简单起见，我们在这里忽略了标点符号和字母大写。

In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])


# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


词元化

下面的tokenize函数将文本行列表（lines）作为输入， 列表中的每个元素是一个文本序列（如一条文本行）。  
每个文本序列又被拆分成一个词元列表，词元（token）是文本的基本单位。  
文本行列表=[文本序列1,文本序列2,...]  
文本序列=[词元1,词元2,...]  
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。  
line.split()是字符串的一个方法，它的核心作用是​​将一个字符串分割成多个子字符串，并将这些子字符串放入一个列表中返回​​

In [3]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]#二维列表[["Hello", "world",....], .....,["Python", "code",.....]]
    elif token == 'char':
        return [list(line) for line in lines]#二维列表[['a','b',...][...][...],...]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
#tokens = tokenize(lines,'char')
for i in range(11):
    print(tokens[i])


['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。  
现在，让我们构建一个字典，通常也叫做词表（vocabulary）， 用来将字符串类型的词元映射到从0开始的数字索引中。  
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计， 得到的统计结果称之为语料（corpus）。  
然后根据每个唯一词元的出现频率，为其分配一个数字索引。 很少出现的词元通常被移除，这可以降低复杂性。  
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“\<unk\>”。  
我们可以选择增加一个列表，用于保存那些被保留的词元， 例如：填充词元（“\<pad\>”）； 序列开始词元（“\<bos\>”）； 序列结束词元（“\<eos\>”）。

In [4]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx#键值对定义，这里将​​词元作为键​​，将其​​索引作为值​​
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):# 情况1：输入是单个词元
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]# 情况2：输入是列表/元组

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)


<pre>python知识点说明  

collections.Counter是 Python 中一个强大的计数工具。  
简单来说，当你把一个列表、字符串或其他可迭代对象交给它时，它会返回一个​​特殊的字典子类对象​​。  
这个对象以原始数据中的元素作为键（key），以该元素出现的次数作为值（value）  

sorted函数：
key参数：自定义排序规则​​
    这是 sorted()函数最强大灵活的部分。它接受一个函数，这个函数会应用于可迭代对象中的​​每一个元素​​，并根据该函数的返回值进行排序。  
reverse参数：控制排序方向​​，一个布尔值参数。
    reverse=False：​​升序​​（从小到大，默认值）。
    reverse=True：​​降序​​（从大到小）
iterable：指定要排序的可迭代对象（如列表、元组、字典等）	
    任何可迭代对象

在Python中，__len__和 __getitem__是两种非常重要的​​魔术方法​​（也称为特殊方法或双下方法）。  
它们的主要作用是让你自定义的类能够像Python内置的数据类型（如列表、字典）一样，支持使用 len()函数获取长度，以及使用中括号 []进行索引操作  

isinstance()是 Python 中一个非常实用的内置函数，主要用于检查一个对象是否属于指定的类型（或其子类）

字典dict的知识
    dict.get(a,b),尝试在dict里面查找a并且返回a的value，如果没有找到，返回b
    dict.items()方法是一个非常有用的工具，主要用于获取字典中所有的键值对，返回一个元组形式(key,value)的view(视图),
    list(my_dict.items()),tuple(my_dict.items())进行转换，转换后的列表或元组就与原始字典“断开连接”了，后续对字典的修改不会影响它们
    .keys()     dict_keys视图       返回字典中所有​​键​​的动态视图
    .values()   dict_values视图     返回字典中所有​​值​​的动态视图。
    .items()    dict_items视图      返回字典中所有​​键值对​​（作为元组）的动态视图

@property是一个装饰器，它能把一个方法“变成”属性来使用，使得可以像访问属性一样调用方法（无需括号）
可以配合@属性名.setter和@属性名.deleter装饰器，创建可读写或可控制的属性
如果只使用@property而不定义setter方法，则可以创建只读属性

Python 中三种主要的成员(方法、属性)访问权限约定
公有成员​​    无下划线前缀               在任何地方都可以自由访问和修改。                                       self.public_attr
​受保护成员​​  单下划线开头 (_member)     这是一种约定，提示该成员仅供类内部或子类使用。​​外部仍可访问​​，但不推荐。        self._protected_attr
​私有成员​​    双下划线开头 (__member)    Python 会进行​​名称改写​​，使其难以直接从类外访问，从而实现一定程度的隔离。      self.__private_attr

魔术方法（Magic Methods），也称为特殊方法或双下方法（Dunder Methods）
    独特的命名规则：身份标识​​
        双下划线的命名方式是一个强烈的视觉信号，旨在将它们与类的普通方法区分开来。当你看到以 __开头和结尾的方法时，就能立刻意识到这是一个会在特定场景下被自动调用的特殊方法
    ​​自动的调用机制：幕后英雄​​
        这是魔术方法最“魔术”的地方。你不需要知道它们的具体方法名并去显式调用。例如，当你使用 print(obj)时，解释器会自动去寻找并执行 obj.__str__()；当你使用 len(obj)时
        ，会自动调用 obj.__len__()。这种机制将复杂的实现细节隐藏起来，让代码变得非常简洁和直观。
    ​​强大的行为定制：自由的灵魂​​
        魔术方法的核心价值在于“定制化”。它允许你定义自己的类该如何应对 Python 的各种内置操作。
            ​​运算符重载​​：你可以通过实现 __add__方法，让你自定义的 Vector类或 Matrix类支持使用 +进行加法运算，使代码更符合数学直觉。
            ​​行为模拟​​：你可以通过实现 __len__和 __getitem__等方法，让你自己的类表现得像一个内置的列表或字典。
            ​​上下文管理​​：通过实现 __enter__和 __exit__方法，可以让你的类支持 with语句，用于安全地获取和释放资源（如文件操作、数据库连接）

我们首先使用时光机器数据集作为语料库来构建词表，然后打印前几个高频词元及其索引。

In [5]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])


[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在，我们可以将每一条文本行转换成一个数字索引列表。

In [6]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])


文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


<pre>整合所有功能

在使用上述函数时，我们将所有功能打包到load_corpus_time_machine函数中， 该函数返回corpus（词元索引列表）和vocab（时光机器语料库的词表）。 我们在这里所做的改变是：

    为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；

    时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的corpus仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [7]:
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    #把字符编码成了数字，全文展平放入到一维list
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)


(170580, 28)

小结

    文本是序列数据的一种最常见的形式之一。

    为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。
